# init

In [1]:
import autogen

# common

In [2]:
config_list = [
    {
        "model": "mixtral 8x instruct 01 7B Q5_K_M",
        "base_url": "http://host.docker.internal:1234/v1",
        "api_type": "openai",
        "api_key": "not-needed",
    }
]
working_dir = "../.data"

# examples

## github research

### create agents

In [ ]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "config_list": config_list,
        "cache_seed": None,
        "temperature": 0.3,
        "timeout": 300,
    },
    system_message="""You are a helpful AI assistant.
    Solve tasks using your coding and language skills.
    In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
        1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
        2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
    Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
    When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
    If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
    """,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": working_dir},
    llm_config={
        "config_list": config_list,
        "temperature": 0.9,
        "timeout": 300,
        "cache_seed": None,
    },
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.  Otherwise, reply CONTINUE, or the reason why the task is not yet solved.""",
)


### start chat

In [ ]:
github_user = "dschartman"

task = f"""
    Use the github api to craft a basic summary of the github user {github_user}.  Format the summary in markdown and write to a local file called "{github_user}_summary.md"
"""

user_proxy.initiate_chat(assistant, message=task)

In [ ]:
import markdown

# Define the path to your Markdown file
markdown_file_path = f"{working_dir}/{github_user}_summary.md"

# Read the Markdown file
with open(markdown_file_path, 'r', encoding='utf-8') as file:
    markdown_content = file.read()

# Convert Markdown to HTML
html_content = markdown.markdown(markdown_content)

# Print the HTML content
print(html_content)

## engineering team

### create agents

In [ ]:
planner = autogen.AssistantAgent(
    name="planner",
    llm_config={"config_list": config_list},
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix.",
)
planner_user = autogen.UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
    code_execution_config={
        "use_docker": False
    },
)

def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]

In [ ]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to: 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result. The planner does not know the conversation between you and the user, unless you share the conversation with the planner.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ],
    },
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "planning",
        "use_docker": False,
    },
    function_map={"ask_planner": ask_planner},
)

### start chat

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="""Suggest a fix to an open good first issue of AutoGen.  AutoGen is a on github here: https://github.com/microsoft/autogen""",
)

## poker game

### create a player

I want to create a poker player that uses a group of agents to decide what to do.  

In [22]:
class PokerPlayer:
    def __init__(self, name, config_list):
        if not name or not isinstance(config_list, list):
            raise ValueError("Name must be a string and config_list must be a list.")
        self.name = name
        self.setup_agents(config_list)
        self.setup_group_chat(config_list)

    def setup_agents(self, config_list):
        self.user_proxy = autogen.UserProxyAgent(
            name=self.name,
            system_message="Your strategy should balance logic with emotion. After considering the game state, format your decision as follows: {'action': '<ACTION>', 'amount': <AMOUNT>, 'chat': '<MESSAGE>'}.",
            llm_config={"config_list": config_list},
        )
        self.logic = autogen.AssistantAgent(
            name="logic",
            system_message="Analyze the current hand and suggest a logical action. Consider the odds and other players' potential hands.",
            llm_config={"config_list": config_list},
        )
        self.emotion = autogen.AssistantAgent(
            name="emotion",
            system_message="Reflect on the game's emotional aspect. How does the current state affect your decision? Share your feelings.",
            llm_config={"config_list": config_list},
        )

    def setup_group_chat(self, config_list):
        self.groupchat = autogen.GroupChat(
            agents=[self.user_proxy, self.logic, self.emotion],
            messages=[],
            max_round=12
        )
        self.manager = autogen.GroupChatManager(
            groupchat=self.groupchat,
            llm_config={"config_list": config_list},
        )

    def decide_move(self, context):
        initial_message = f"{self.name}, it's your turn. Here's the game state: {context}. How will you play your hand?"
        self.user_proxy.initiate_chat(self.manager, message=initial_message)

    # Implement the method to process and integrate decisions from logic and emotion agents
    def process_decision(self):
        # Placeholder for decision processing logic
        pass


### run a hand

In [23]:
player = PokerPlayer("Karl", config_list)

In [24]:
import json

game_state = {
    'players': [
        {'name': 'David', 'seat': 0},
        {'name': 'Susie', 'seat': 1},
        {'name': 'Karl', 'seat': 2},
        {'name': 'Stevie', 'seat': 3},
    ],
    'dealer_position': 1,
    'small_blind_amount': 50,
    'big_blind_amount': 100,
    'blind_level': 1,
    'next_blind_increase': '2024-02-29T13:00:00',
    'community_cards': ['10C', 'JH', 'QC', '', ''],
    'side_pots': [], 
    'current_action_to': 2,
    'hole_cards': ['AS', 'KD'],
    'action_history': [
        {'player_seat': 2, 'action': 'post_small_blind', 'amount': 50, 'stack': 850, 'pot': 50},
        {'player_seat': 3, 'action': 'post_big_blind', 'amount': 100, 'stack': 800, 'pot': 150},
        {'player_seat': 0, 'action': 'raise', 'amount': 200, 'stack': 700, 'pot': 350},
        {'player_seat': 1, 'action': 'fold', 'amount': 0, 'stack': 900, 'pot': 350},
        {'player_seat': 2, 'action': 'call', 'amount': 150, 'stack': 700, 'pot': 500},
        {'player_seat': 3, 'action': 'fold', 'amount': 150, 'stack': 800, 'pot': 500},
    ],
    'hand_state': 'flop',
    'chat_messages': [
        {'player_seat': 3, 'message': 'Nice hand!', 'timestamp': '2024-02-29T12:34:56'},
        {'player_seat': 1, 'message': 'Thanks!', 'timestamp': '2024-02-29T12:35:10'},
    ],
}

player.decide_move(context=json.dumps(game_state))

Karl (to chat_manager):

Karl, it's your turn. Here's the game state: {"players": [{"name": "David", "seat": 0}, {"name": "Susie", "seat": 1}, {"name": "Karl", "seat": 2}, {"name": "Stevie", "seat": 3}], "dealer_position": 1, "small_blind_amount": 50, "big_blind_amount": 100, "blind_level": 1, "next_blind_increase": "2024-02-29T13:00:00", "community_cards": ["10C", "JH", "QC", "", ""], "side_pots": [], "current_action_to": 2, "hole_cards": ["AS", "KD"], "action_history": [{"player_seat": 2, "action": "post_small_blind", "amount": 50, "stack": 850, "pot": 50}, {"player_seat": 3, "action": "post_big_blind", "amount": 100, "stack": 800, "pot": 150}, {"player_seat": 0, "action": "raise", "amount": 200, "stack": 700, "pot": 350}, {"player_seat": 1, "action": "fold", "amount": 0, "stack": 900, "pot": 350}, {"player_seat": 2, "action": "call", "amount": 150, "stack": 700, "pot": 500}, {"player_seat": 3, "action": "fold", "amount": 150, "stack": 800, "pot": 500}], "hand_state": "flop", "chat_m

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:

### Conversation:
Karl: I see I have an Ace and a King. The flop is 10C, JH, QC. I'm in the small blind position and have already contributed 50 to the pot. David raised it by 200 after Stevie folded, Susie then folded, and I called. Now it's my turn again with two players left: David and myself. My stack is at 700 now, while David has 700 as well. What should be my next move?
logic: Given your hand strength and the current pot size, you might consider raising or re-raising to build the pot further and extract value from David. Keep in mind that he raised initially, suggesting a stronger hand, but with two cards still hidden and only two players left, it's worth trying to take control of the betting round.
emotion: I agree with logic; this is an exciting opportunity! After all, we have top pair and a decent kicker here, so let's try to make the most of it. Howev

logic (to chat_manager):

Based on the given game state, I'm Karl and my hole cards are A♠K♦. There has been a raise from David (seat 0) to 200 over the big blind of 100, and I called the additional 100 to put me in for 250 total (50 small blind + 150 call). The flop came out J♥Q♣10♣.

At this point, with top pair and a decent kicker, I'm likely ahead of David's range but still want to exercise caution given the potential for strong draws on the board. With two players in the hand, there is a side pot of 450 (200 from David + 100 big blind + 150 call).

Considering that I have top pair and an above-average kicker, my options are:

1. Check and evaluate future betting rounds based on board texture and opponent actions. This could allow me to keep the pot smaller while still trying to extract value from potentially weaker hands.
2. Make a continuation bet (c-bet) here, which can help take down the pot immediately or build it further if called. The size of my c-bet could range between 1/3

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:
 What do you think?



emotion (to chat_manager):

As Karl, I check given the board texture and presence of straight and flush draws. This allows me to maintain control over future betting rounds while minimizing potential loss in case of a stronger hand or draw.

--------------------------------------------------------------------------------


...

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:



logic (to chat_manager):

As Karl, I check the flop with my top pair and decent kicker (A♠K♦) given the board texture and potential for strong draws (J♥Q♣10♣). This action allows me to maintain control over future betting rounds while minimizing potential loss in case of a stronger hand or draw.

Here is the updated game state:
{"players": [{"name": "David", "seat": 0}, {"name": "Susie", "seat": 1}, {"name": "Karl", "seat": 2}, {"name": "Stevie", "seat": 3}], "dealer_position": 1, "small_blind_amount": 50, "big_blind_amount": 100, "blind_level": 1, "next_blind_increase": "2024-02-29T13:00:00", "community_cards": ["10C", "JH", "QC", "", ""], "side_pots": [{"pot_amount": 450, "players_in_pot": [{"player_seat": 0, "stack": 700}, {"player_seat": 2, "stack": 700}]}], "current_action_to": 0, "hole_cards": ["AS", "KD"], "action_history": [{"player_seat": 2, "action": "post_small_blind", "amount": 50, "stack": 850, "pot": 50}, {"player_seat": 3, "action": "post_big_blind", "amount": 100, "stac

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:



emotion (to chat_manager):

As Karl, I check given the board texture and presence of strong draws (J♥Q♣10♣). This action allows me to maintain control over future betting rounds while minimizing potential loss in case of a stronger hand or draw.

--------------------------------------------------------------------------------


GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:

